In [1]:
### imports
import warnings
warnings.simplefilter('ignore')
import os
import numpy as np
import pandas as pd
import joblib
import yaml

In [2]:
def make_main_synthetic_results_table(config_path, results_dir, metric_funcs):

    with open(config_path) as f:
        config = yaml.safe_load(f)

    setup_options = config['setup-options']

    for clever in setup_options['clever']:
        resd = {}
        for dgp_version in setup_options['dgp-version']:
            resd[dgp_version] = {}
            for n in setup_options['n']:
                lambda_l2_h = .1/n**(.9)
                # print(lambda_l2_h)
                nkey = f'$n={n}$'
                resd[dgp_version][nkey] = {}
                for iv_strength in setup_options['iv-strength']:
                    setup = {
                        'clever': clever,
                        'dgp-version': dgp_version,
                        'n': n,
                        'iv-strength': iv_strength,
                    }
                    results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
                    results_path = os.path.join(results_dir, results_fname)
                    true, results = joblib.load(results_path)
                    ivkey = f'$\rho={iv_strength}$'
                    resd[dgp_version][nkey][ivkey] = {}
                    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                        # get array of results for given method over experiment replications
                        data = np.array([r[it] for r in results])
                        resd[dgp_version][nkey][ivkey][method] = {
                            metric_: metric_func_(data, true, method)
                            for metric_, metric_func_ in metric_funcs.items()
                        }
                        resd[dgp_version][nkey][ivkey] = pd.DataFrame(resd[dgp_version][nkey][ivkey])
                resd[dgp_version][nkey] = pd.concat(resd[dgp_version][nkey])
            resd[dgp_version] = pd.concat(resd[dgp_version])

        display(pd.concat(resd).unstack(level=3))
        print(pd.concat(resd).unstack(level=3).to_latex(bold_rows=True, multirow=True,
                                                        multicolumn=True, escape=False,
                                                        column_format='lll||lll|lll|lll|lll|',
                                                        multicolumn_format='c|'))


def get_cov(data, true, method):
    if method in ['dr', 'tmle']:
        return f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
    else:
        return 'NA'


def get_rmse(data, true, method):
    rmse = np.sqrt(np.mean((data[:, 0] - true) ** 2))
    return f'{rmse:.3f}'


def get_bias(data, true, method):
    abs_bias = np.abs(np.mean((data[:, 0] - true)))
    return f'{abs_bias:.3f}'


def get_normalized_rmse(data, true, method):
    rmse = np.sqrt(np.mean((data[:, 0] - true) ** 2))
    norm_rmse = rmse / np.abs(true)
    return f'{norm_rmse:.3f}'


def get_normalized_bias(data, true, method):
    abs_bias = np.abs(np.mean((data[:, 0] - true)))
    norm_abs_bias = abs_bias / np.abs(true)
    return f'{norm_abs_bias:.3f}'
    
    

In [3]:
# results for strong instruments experiment

make_main_synthetic_results_table(
    config_path='experiment-configs/strong-instruments-config.yaml',
    results_dir='results/strong-instruments',
    metric_funcs={
        'cov': get_cov,
        'rmse': get_rmse,
        'bias': get_bias,
    },
)

dr               tmle               ipw         \
                            cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $n=500$  $\rho=0.2$  92  0.415  0.012   85  2.669  0.408  NA  0.775   
                 $\rho=0.5$  91  0.169  0.003   91  0.166  0.008  NA  0.229   
                 $\rho=0.7$  89  0.122  0.007   88  0.124  0.006  NA  0.146   
        $n=1000$ $\rho=0.2$  97  0.337  0.025   95  1.025  0.221  NA  0.838   
                 $\rho=0.5$  94  0.110  0.015   96  0.106  0.017  NA  0.130   
                 $\rho=0.7$  94  0.082  0.007   95  0.080  0.008  NA  0.084   
        $n=2000$ $\rho=0.2$  97  0.274  0.010   97  0.259  0.065  NA  0.930   
                 $\rho=0.5$  95  0.079  0.013   94  0.079  0.014  NA  0.123   
                 $\rho=0.7$  96  0.054  0.008   96  0.053  0.008  NA  0.086   
2dpoly  $n=500$  $\rho=0.2$  95  0.528  0.038   88  1.082  0.191  NA  1.196   
                 $\rho=0.5$  94  0.241  0.000   94  0.237  0.007  NA  0.338   
                 $\rho=0.7$  92  0.209  0.002   92  0.210  0.002  NA  0.301   
        $n=1000$ $\rho=0.2$  95  0.486  0.080   96  0.598  0.181  NA  1.607   
                 $\rho=0.5$  95  0.168  0.014   95  0.166  0.017  NA  0.271   
                 $\rho=0.7$  97  0.131  0.003   97  0.129  0.003  NA  0.197   
        $n=2000$ $\rho=0.2$  96  0.389  0.005   95  0.349  0.049  NA  1.561   
                 $\rho=0.5$  98  0.107  0.009   98  0.108  0.009  NA  0.222   
                 $\rho=0.7$  96  0.101  0.005   96  0.100  0.005  NA  0.202   
sigmoid $n=500$  $\rho=0.2$  91  0.423  0.010   80  2.059  0.335  NA  0.792   
                 $\rho=0.5$  91  0.152  0.009   92  0.151  0.003  NA  0.251   
                 $\rho=0.7$  92  0.098  0.010   90  0.100  0.011  NA  0.124   
        $n=1000$ $\rho=0.2$  96  0.324  0.001   96  0.926  0.185  NA  0.710   
                 $\rho=0.5$  95  0.094  0.008   96  0.090  0.010  NA  0.129   
                 $\rho=0.7$  93  0.066  0.003   93  0.066  0.003  NA  0.075   
        $n=2000$ $\rho=0.2$  94  0.299  0.028   96  0.297  0.083  NA  0.816   
                 $\rho=0.5$  95  0.068  0.007   96  0.066  0.009  NA  0.126   
                 $\rho=0.7$  96  0.045  0.003   96  0.045  0.003  NA  0.071   
sin     $n=500$  $\rho=0.2$  92  0.424  0.008   82  2.060  0.343  NA  0.464   
                 $\rho=0.5$  90  0.143  0.020   91  0.142  0.013  NA  0.171   
                 $\rho=0.7$  89  0.097  0.014   88  0.100  0.015  NA  0.102   
        $n=1000$ $\rho=0.2$  95  0.333  0.019   98  0.920  0.202  NA  0.401   
                 $\rho=0.5$  93  0.090  0.002   96  0.085  0.004  NA  0.107   
                 $\rho=0.7$  95  0.065  0.002   95  0.064  0.002  NA  0.071   
        $n=2000$ $\rho=0.2$  95  0.287  0.031   94  0.289  0.084  NA  0.338   
                 $\rho=0.5$  96  0.063  0.005   96  0.061  0.007  NA  0.076   
                 $\rho=0.7$  95  0.042  0.002   95  0.041  0.002  NA  0.052   

                                   direct                
                              bias    cov   rmse   bias  
abs     $n=500$  $\rho=0.2$  0.147     NA  0.281  0.001  
                 $\rho=0.5$  0.008     NA  0.110  0.004  
                 $\rho=0.7$  0.005     NA  0.089  0.006  
        $n=1000$ $\rho=0.2$  0.042     NA  0.236  0.017  
                 $\rho=0.5$  0.022     NA  0.082  0.001  
                 $\rho=0.7$  0.005     NA  0.066  0.003  
        $n=2000$ $\rho=0.2$  0.118     NA  0.203  0.035  
                 $\rho=0.5$  0.018     NA  0.074  0.009  
                 $\rho=0.7$  0.001     NA  0.054  0.007  
2dpoly  $n=500$  $\rho=0.2$  0.094     NA  0.448  0.043  
                 $\rho=0.5$  0.056     NA  0.208  0.021  
                 $\rho=0.7$  0.037     NA  0.188  0.002  
        $n=1000$ $\rho=0.2$  0.114     NA  0.331  0.016  
                 $\rho=0.5$  0.131     NA  0.162  0.008  
                 $\rho=0.7$  0.032     NA  0.123  0.018  
        $n=2000$ $\rho=0.2$  0.334     N

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
ho=0.2$} & 92 & 0.415 & 0.012 & 85 & 2.669 & 0.408 & NA & 0.775 & 0.147 & NA & 0.281 & 0.001 \\
ho=0.5$} & 91 & 0.169 & 0.003 & 91 & 0.166 & 0.008 & NA & 0.229 & 0.008 & NA & 0.110 & 0.004 \\
ho=0.7$} & 89 & 0.122 & 0.007 & 88 & 0.124 & 0.006 & NA & 0.146 & 0.005 & NA & 0.089 & 0.006 \\
\cline{2-15}
ho=0.2$} & 97 & 0.337 & 0.025 & 95 & 1.025 & 0.221 & NA & 0.838 & 0.042 & NA & 0.236 & 0.017 \\
ho=0.5$} & 94 & 0.110 & 0.015 & 96 & 0.106 & 0.017 & NA & 0.130 & 0.022 & NA & 0.082 & 0.001 \\
ho=0.7$} & 94 & 0.082 & 0.007 & 95 & 0.080 & 0.008 & NA & 0.084 & 0.005 & NA & 0.066 & 0.003 \\
\cline{2-15}
ho=0.2$} & 97 & 0.274 & 0.010 & 97 & 0.259 & 0.065 & NA & 0.930 & 0.118 & NA & 0.203 & 0.035 \\
ho=0.5$} & 95 & 0.079 & 0.013 & 94

In [4]:
# results for clever instruments experiment

make_main_synthetic_results_table(
    config_path='experiment-configs/clever-instruments-config.yaml',
    results_dir='results/clever-instruments',
    metric_funcs={
        'cov': get_cov,
        'rmse': get_rmse,
        'bias': get_bias,
    },
)

dr               tmle               ipw         \
                            cov   rmse   bias  cov   rmse   bias cov   rmse   
abs     $n=500$  $\rho=0.2$  82  0.315  0.144   87  1.766  0.301  NA  0.775   
                 $\rho=0.5$  90  0.167  0.011   92  0.168  0.015  NA  0.229   
        $n=1000$ $\rho=0.2$  93  0.266  0.044   96  1.044  0.189  NA  0.838   
                 $\rho=0.5$  95  0.115  0.025   96  0.112  0.024  NA  0.130   
        $n=2000$ $\rho=0.2$  92  0.252  0.046   98  0.285  0.022  NA  0.930   
                 $\rho=0.5$  92  0.082  0.018   92  0.081  0.015  NA  0.123   
2dpoly  $n=500$  $\rho=0.2$  90  0.482  0.147   87  0.980  0.156  NA  1.196   
                 $\rho=0.5$  95  0.234  0.006   95  0.232  0.010  NA  0.338   
        $n=1000$ $\rho=0.2$  94  0.435  0.075   93  0.559  0.143  NA  1.607   
                 $\rho=0.5$  96  0.161  0.017   96  0.159  0.017  NA  0.271   
        $n=2000$ $\rho=0.2$  98  0.323  0.005   98  0.335  0.017  NA  1.561   
                 $\rho=0.5$  97  0.110  0.013   97  0.109  0.012  NA  0.222   
sigmoid $n=500$  $\rho=0.2$  81  0.310  0.163   81  1.196  0.201  NA  0.792   
                 $\rho=0.5$  91  0.143  0.000   91  0.147  0.004  NA  0.251   
        $n=1000$ $\rho=0.2$  94  0.255  0.028   98  0.775  0.145  NA  0.710   
                 $\rho=0.5$  95  0.094  0.020   96  0.091  0.019  NA  0.129   
        $n=2000$ $\rho=0.2$  89  0.286  0.046   99  0.309  0.024  NA  0.816   
                 $\rho=0.5$  92  0.072  0.016   93  0.070  0.013  NA  0.126   
sin     $n=500$  $\rho=0.2$  82  0.348  0.183   82  1.084  0.157  NA  0.464   
                 $\rho=0.5$  91  0.135  0.001   92  0.138  0.003  NA  0.171   
        $n=1000$ $\rho=0.2$  93  0.262  0.044   96  1.144  0.199  NA  0.401   
                 $\rho=0.5$  95  0.087  0.021   97  0.084  0.019  NA  0.107   
        $n=2000$ $\rho=0.2$  89  0.256  0.047   98  0.293  0.025  NA  0.338   
                 $\rho=0.5$  92  0.067  0.013   93  0.065  0.011  NA  0.076   

                                   direct                
                              bias    cov   rmse   bias  
abs     $n=500$  $\rho=0.2$  0.147     NA  0.535  0.513  
                 $\rho=0.5$  0.008     NA  0.115  0.046  
        $n=1000$ $\rho=0.2$  0.042     NA  0.536  0.520  
                 $\rho=0.5$  0.022     NA  0.094  0.046  
        $n=2000$ $\rho=0.2$  0.118     NA  0.514  0.493  
                 $\rho=0.5$  0.018     NA  0.073  0.040  
2dpoly  $n=500$  $\rho=0.2$  0.094     NA  0.553  0.465  
                 $\rho=0.5$  0.056     NA  0.208  0.028  
        $n=1000$ $\rho=0.2$  0.114     NA  0.547  0.487  
                 $\rho=0.5$  0.131     NA  0.139  0.034  
        $n=2000$ $\rho=0.2$  0.335     NA  0.479  0.434  
                 $\rho=0.5$  0.106     NA  0.106  0.030  
sigmoid $n=500$  $\rho=0.2$  0.210     NA  0.552  0.530  
                 $\rho=0.5$  0.020     NA  0.099  0.056  
        $n=1000$ $\rho=0.2$  0.021     NA  0.553  0.538  
                 $\rho=0.5$  0.024     NA  0.084  0.046  
        $n=2000$ $\rho=0.2$  0.064     NA  0.513  0.488  
                 $\rho=0.5$  0.016     NA  0.074  0.044  
sin     $n=500$  $\rho=0.2$  0.101     NA  0.573  0.543  
                 $\rho=0.5$  0.038     NA  0.101  0.057  
        $n=1000$ $\rho=0.2$  0.054     NA  0.556  0.543  
                 $\rho=0.5$  0.030     NA  0.077  0.043  
        $n=2000$ $\rho=0.2$  0.035     NA  0.523  0.493  
                 $\rho=0.5$  0.017     NA  0.057  0.032

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias & cov & rmse & bias \\
\midrule
ho=0.2$} & 82 & 0.315 & 0.144 & 87 & 1.766 & 0.301 & NA & 0.775 & 0.147 & NA & 0.535 & 0.513 \\
ho=0.5$} & 90 & 0.167 & 0.011 & 92 & 0.168 & 0.015 & NA & 0.229 & 0.008 & NA & 0.115 & 0.046 \\
\cline{2-15}
ho=0.2$} & 93 & 0.266 & 0.044 & 96 & 1.044 & 0.189 & NA & 0.838 & 0.042 & NA & 0.536 & 0.520 \\
ho=0.5$} & 95 & 0.115 & 0.025 & 96 & 0.112 & 0.024 & NA & 0.130 & 0.022 & NA & 0.094 & 0.046 \\
\cline{2-15}
ho=0.2$} & 92 & 0.252 & 0.046 & 98 & 0.285 & 0.022 & NA & 0.930 & 0.118 & NA & 0.514 & 0.493 \\
ho=0.5$} & 92 & 0.082 & 0.018 & 92 & 0.081 & 0.015 & NA & 0.123 & 0.018 & NA & 0.073 & 0.040 \\
\cline{1-15} \cline{2-15}
ho=0.2$} & 90 & 0.482 & 0.147 & 87 & 0.980 & 0.156 & NA & 1.196 & 0.094 & NA & 0.553 & 0.465 \\
ho=0.5$}

In [5]:
# results for weak instruments experiment

make_main_synthetic_results_table(
    config_path='experiment-configs/weak-instruments-config.yaml',
    results_dir='results/weak-instruments',
    metric_funcs={
        'cov': get_cov,
        '$\\frac{\\textbf{rmse}}{|\\theta^*|}$': get_normalized_rmse,
        '$\\frac{\\textbf{bias}}{|\\theta^*|}$': get_normalized_bias,
    },
)

dr                                     \
                             cov $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$  93                              1.876   
                 $\rho=0.1$   97                              0.982   
       $n=20000$ $\rho=0.05$  95                              2.009   
                 $\rho=0.1$   96                              0.438   

                                                                tmle  \
                             $\frac{\textbf{bias}}{|\theta^*|}$  cov   
2dpoly $n=2000$  $\rho=0.05$                              0.256   99   
                 $\rho=0.1$                               0.043   96   
       $n=20000$ $\rho=0.05$                              0.125  100   
                 $\rho=0.1$                               0.022   96   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                            104.116   
                 $\rho=0.1$                               2.237   
       $n=20000$ $\rho=0.05$                             12.320   
                 $\rho=0.1$                               0.368   

                                                                ipw  \
                             $\frac{\textbf{bias}}{|\theta^*|}$ cov   
2dpoly $n=2000$  $\rho=0.05$                             12.105  NA   
                 $\rho=0.1$                               0.237  NA   
       $n=20000$ $\rho=0.05$                              1.311  NA   
                 $\rho=0.1$                               0.003  NA   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                              8.106   
                 $\rho=0.1$                               4.414   
       $n=20000$ $\rho=0.05$                              9.740   
                 $\rho=0.1$                               1.917   

                                                                direct  \
                             $\frac{\textbf{bias}}{|\theta^*|}$    cov   
2dpoly $n=2000$  $\rho=0.05$                              0.636     NA   
                 $\rho=0.1$                               0.046     NA   
       $n=20000$ $\rho=0.05$                              1.030     NA   
                 $\rho=0.1$                               0.041     NA   

                                                                 \
                             $\frac{\textbf{rmse}}{|\theta^*|}$   
2dpoly $n=2000$  $\rho=0.05$                              0.836   
                 $\rho=0.1$                               0.408   
       $n=20000$ $\rho=0.05$                              0.285   
                 $\rho=0.1$                               0.120   

                                                                 
                             $\frac{\textbf{bias}}{|\theta^*|}$  
2dpoly $n=2000$  $\rho=0.05$                              0.512  
                 $\rho=0.1$                               0.079  
       $n=20000$ $\rho=0.05$                              0.016  
                 $\rho=0.1$                               0.012

\begin{tabular}{lll||lll|lll|lll|lll|}
\toprule
 &  &  & \multicolumn{3}{c|}{dr} & \multicolumn{3}{c|}{tmle} & \multicolumn{3}{c|}{ipw} & \multicolumn{3}{c|}{direct} \\
 &  &  & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ & cov & $\frac{\textbf{rmse}}{|\theta^*|}$ & $\frac{\textbf{bias}}{|\theta^*|}$ \\
\midrule
ho=0.05$} & 93 & 1.876 & 0.256 & 99 & 104.116 & 12.105 & NA & 8.106 & 0.636 & NA & 0.836 & 0.512 \\
ho=0.1$} & 97 & 0.982 & 0.043 & 96 & 2.237 & 0.237 & NA & 4.414 & 0.046 & NA & 0.408 & 0.079 \\
\cline{2-15}
ho=0.05$} & 95 & 2.009 & 0.125 & 100 & 12.320 & 1.311 & NA & 9.740 & 1.030 & NA & 0.285 & 0.016 \\
ho=0.1$} & 96 & 0.438 & 0.022 & 96 & 0.368 & 0.003 & NA & 1.917 & 0.041 & NA & 0.120 & 0.012 \\
\cline{1-15} \cline{2-15}
\bottomrule
\end{tabular}



In [6]:
# results for partially linear IV experiments

config_path = 'experiment-configs/pliv-config.yaml'
results_dir = 'results/pliv'

def get_signed_bias(data, true, n):
    bias = np.mean((data[:, 0] - true))
    return f'{bias:.3f}'


def get_root_n_std(data, true, n):
    estimator_std = np.std(data[:, 0])
    return estimator_std * (n ** 0.5)


metric_funcs = {
    'bias': get_signed_bias,
    'root-n-std': get_root_n_std,
    # 'abs-bias': get_bias,
    # 'rmse': get_rmse,
}


with open(config_path) as f:
    config = yaml.safe_load(f)

setup_options = config['setup-options']

for clever in setup_options['clever']:
    print(f'clever = {clever}')
    resd = {}
    for dgp_version in setup_options['dgp-version']:
        print(f' - DGP version = {dgp_version}')
        resd[dgp_version] = {}
        for n in setup_options['n']:
            lambda_l2_h = .1/n**(.9)
            # print(lambda_l2_h)
            nkey = f'$n={n}$'
            resd[dgp_version][nkey] = {}
            assert len(setup_options['iv-strength'])== 1
            iv_strength = setup_options['iv-strength'][0]
            setup = {
                'clever': clever,
                'dgp-version': dgp_version,
                'n': n,
                'iv-strength': iv_strength,
            }
            results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
            results_path = os.path.join(results_dir, results_fname)
            true, results = joblib.load(results_path)
            resd[dgp_version][nkey] = {}
            for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                # get array of results for given method over experiment replications
                data = np.array([r[it] for r in results])
                # print(f'result for method={method}, n={n}:')
                # print(f'true = {true}')
                # display(pd.Series(data[:, 0]).describe())
                resd[dgp_version][nkey][method] = {
                    metric_: metric_func_(data, true, n)
                    for metric_, metric_func_ in metric_funcs.items()
                }
                resd[dgp_version][nkey] = pd.DataFrame(resd[dgp_version][nkey])
        resd[dgp_version] = pd.concat(resd[dgp_version])

    display(pd.concat(resd).unstack(level=2))
    print(pd.concat(resd).unstack(level=2).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='ll||ll|ll|ll|ll|',
                                                    multicolumn_format='c|'))



clever = True
 - DGP version = DGP-1


dr                 tmle                   ipw              \
                  bias root-n-std      bias    root-n-std    bias  root-n-std   
DGP-1 $n=100$   -0.552   8.556239     0.325    265.280776  -0.919   20.987186   
      $n=500$   -0.192  19.777352     4.177   1698.433666  -1.911  158.834184   
      $n=1000$  -0.377  34.116817    -1.120     339.69932  -0.317  331.161631   
      $n=2000$  -0.390  64.818547  -186.636  84374.681066   0.364  755.656202   

                direct             
                  bias root-n-std  
DGP-1 $n=100$   -0.553   8.557271  
      $n=500$   -0.195  17.651085  
      $n=1000$  -0.278  25.654456  
      $n=2000$  -0.144  33.388127

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & bias & root-n-std & bias & root-n-std & bias & root-n-std & bias & root-n-std \\
\midrule
\multirow[t]{4}{*}{\textbf{DGP-1}} & \textbf{$n=100$} & -0.552 & 8.556239 & 0.325 & 265.280776 & -0.919 & 20.987186 & -0.553 & 8.557271 \\
\textbf{} & \textbf{$n=500$} & -0.192 & 19.777352 & 4.177 & 1698.433666 & -1.911 & 158.834184 & -0.195 & 17.651085 \\
\textbf{} & \textbf{$n=1000$} & -0.377 & 34.116817 & -1.120 & 339.699320 & -0.317 & 331.161631 & -0.278 & 25.654456 \\
\textbf{} & \textbf{$n=2000$} & -0.390 & 64.818547 & -186.636 & 84374.681066 & 0.364 & 755.656202 & -0.144 & 33.388127 \\
\cline{1-10}
\bottomrule
\end{tabular}

clever = False
 - DGP version = DGP-1


dr                 tmle                   ipw              \
                  bias root-n-std      bias    root-n-std    bias  root-n-std   
DGP-1 $n=100$   -0.563   8.658977     1.470    292.865895  -0.919   20.987186   
      $n=500$   -0.624  23.597716     2.224   1623.295634  -1.911  158.834184   
      $n=1000$  -0.572  35.804127    -1.335    356.547114  -0.317  331.161631   
      $n=2000$  -0.402  78.384729  -208.770  95582.631729   0.364  755.656202   

                direct             
                  bias root-n-std  
DGP-1 $n=100$   -0.563   8.578198  
      $n=500$   -0.616  21.917372  
      $n=1000$  -0.534  26.987407  
      $n=2000$  -0.165  45.283072

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & bias & root-n-std & bias & root-n-std & bias & root-n-std & bias & root-n-std \\
\midrule
\multirow[t]{4}{*}{\textbf{DGP-1}} & \textbf{$n=100$} & -0.563 & 8.658977 & 1.470 & 292.865895 & -0.919 & 20.987186 & -0.563 & 8.578198 \\
\textbf{} & \textbf{$n=500$} & -0.624 & 23.597716 & 2.224 & 1623.295634 & -1.911 & 158.834184 & -0.616 & 21.917372 \\
\textbf{} & \textbf{$n=1000$} & -0.572 & 35.804127 & -1.335 & 356.547114 & -0.317 & 331.161631 & -0.534 & 26.987407 \\
\textbf{} & \textbf{$n=2000$} & -0.402 & 78.384729 & -208.770 & 95582.631729 & 0.364 & 755.656202 & -0.165 & 45.283072 \\
\cline{1-10}
\bottomrule
\end{tabular}



In [7]:
# results for average derivative on DGP in CCT paper

config_path = 'experiment-configs/cct-config.yaml'
results_dir = 'results/cct'

def get_mean(data, true, n):
    return data[:, 0].mean()


def get_std(data, true, n):
    return data[:, 0].std()


metric_funcs = {
    'mean': get_mean,
    'std': get_std,
}


with open(config_path) as f:
    config = yaml.safe_load(f)

setup_options = config['setup-options']

for clever in setup_options['clever']:
    print(f'clever = {clever}')
    resd = {}
    for dgp_version in setup_options['dgp-version']:
        print(f' - DGP version = {dgp_version}')
        resd[dgp_version] = {}
        for n in setup_options['n']:
            lambda_l2_h = .1/n**(.9)
            # print(lambda_l2_h)
            nkey = f'$n={n}$'
            resd[dgp_version][nkey] = {}
            assert len(setup_options['iv-strength'])== 1
            iv_strength = setup_options['iv-strength'][0]
            setup = {
                'clever': clever,
                'dgp-version': dgp_version,
                'n': n,
                'iv-strength': iv_strength,
            }
            results_fname = '__'.join(f'{k_}_{setup[k_]}' for k_ in setup_options.keys()) + '.jbl'
            results_path = os.path.join(results_dir, results_fname)
            true, results = joblib.load(results_path)
            resd[dgp_version][nkey] = {}
            for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
                # get array of results for given method over experiment replications
                data = np.array([r[it] for r in results])
                # print(f'result for method={method}, n={n}:')
                # print(f'true = {true}')
                # display(pd.Series(data[:, 0]).describe())
                resd[dgp_version][nkey][method] = {
                    metric_: metric_func_(data, true, n)
                    for metric_, metric_func_ in metric_funcs.items()
                }
                resd[dgp_version][nkey] = pd.DataFrame(resd[dgp_version][nkey])
        resd[dgp_version] = pd.concat(resd[dgp_version])

    display(pd.concat(resd).unstack(level=2))
    print(pd.concat(resd).unstack(level=2).to_latex(bold_rows=True, multirow=True,
                                                    multicolumn=True, escape=False,
                                                    column_format='ll||ll|ll|ll|ll|',
                                                    multicolumn_format='c|'))



clever = True
 - DGP version = MC2


dr                tmle                 ipw            \
                  mean       std      mean       std      mean       std   
MC2 $n=1000$  1.150949  0.158529  1.173178  0.143926  1.546083  4.798609   
    $n=5000$  1.066365  0.107496  1.133543  0.078166  1.750267  5.529226   

                direct            
                  mean       std  
MC2 $n=1000$  1.294742  0.081570  
    $n=5000$  1.276180  0.043668

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & mean & std & mean & std & mean & std & mean & std \\
\midrule
\multirow[t]{2}{*}{\textbf{MC2}} & \textbf{$n=1000$} & 1.150949 & 0.158529 & 1.173178 & 0.143926 & 1.546083 & 4.798609 & 1.294742 & 0.081570 \\
\textbf{} & \textbf{$n=5000$} & 1.066365 & 0.107496 & 1.133543 & 0.078166 & 1.750267 & 5.529226 & 1.276180 & 0.043668 \\
\cline{1-10}
\bottomrule
\end{tabular}

clever = False
 - DGP version = MC2


dr                tmle                 ipw            \
                  mean       std      mean       std      mean       std   
MC2 $n=1000$  1.161099  0.145584  1.157694  0.139463  1.546083  4.798609   
    $n=5000$  1.117925  0.109414  1.114984  0.077220  1.750267  5.529226   

                direct            
                  mean       std  
MC2 $n=1000$  1.179854  0.158979  
    $n=5000$  1.111690  0.102274

\begin{tabular}{ll||ll|ll|ll|ll|}
\toprule
 &  & \multicolumn{2}{c|}{dr} & \multicolumn{2}{c|}{tmle} & \multicolumn{2}{c|}{ipw} & \multicolumn{2}{c|}{direct} \\
 &  & mean & std & mean & std & mean & std & mean & std \\
\midrule
\multirow[t]{2}{*}{\textbf{MC2}} & \textbf{$n=1000$} & 1.161099 & 0.145584 & 1.157694 & 0.139463 & 1.546083 & 4.798609 & 1.179854 & 0.158979 \\
\textbf{} & \textbf{$n=5000$} & 1.117925 & 0.109414 & 1.114984 & 0.077220 & 1.750267 & 5.529226 & 1.111690 & 0.102274 \\
\cline{1-10}
\bottomrule
\end{tabular}

